## Truth Commission Agent 

This notebook implements the prototype of a multi-agent system that simulates an investigation conducted by a [truth commission](https://www.britannica.com/topic/truth-commission) in the aftermath of a violent civil war. In particular, it aims to illustrate how perpetrators of rights violations might navigate this vital aspect in processes of post-conflict reconciliation and political transition. 

The simulation takes place in _Pacifica_, which is a fictional country that has been torn apart after years of civil war that culminated in a violent military coup. The new leaders of Pacifica have faced international pressure to reckon with the atrocities that took place throughout the conflict. In an unprecedented act, they established the _Commission for National Reflection_ (CNR)---an official truth commission tasked with investigating human rights abuses, documenting an impartial and comprehensive historical record from all sides, and facilitating national reconciliation. In this phase of the CNR's investigation, representatives from the `Military Junta`, `Overthrown Government`, and `Democratic Opposition` have been invited to piece together a detailed account of the events that preceded this critical juncture in Pacifica's recent history. By engaging these potential perpetrators and  in dialogue with one another, the CNR seeks to uncover what happened in order to explore the decision-making processes that took place throughout the conflict and prevent this from happening again. 

In this dialogue simulation, representatives from each group provide a brief testimony on what happened during the week leading up to the military coup. They take turns recounting these events on a day-by-day basis, with minimal intervention from the CNR. The CNR representative initiates the discussion, announces when the `Military Junta`, `Overthrown Government`, and `Democratic Opposition` agents should move on to discuss the next day, and concludes. 

This notebook contains code that does the following:

- Generates 3 LLM-based agents representing the `Democratic Opposition`, `Military Junta`, and `Overthrown Government` with their own unique sets of goals, beliefs, and behavioral tendencies that will guide their actions in the simulation
- Simulates an interaction between the agents, in which agents recount detailed information about what happened in the week that culminated in the military coup  
- Saves all transcripts and relevant information to be used for preliminary assessments of the prototype

Overall, this prototype offers initial insights into the processes of post-conflict reconciliation and political transition. It also showcases the potential of using Large Language Model (LLM)-based multi-agent systems for political science research in a novel context.

### Import dependencies

In [1]:
import numpy as np
import pandas as pd 
import os

from collections import OrderedDict
from typing import Callable, List
import random
import functools

import tenacity 
from langchain.output_parsers import RegexParser
from langchain.prompts import PromptTemplate 
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)
from langchain_openai import ChatOpenAI

### Set environment variables for API keys

In [2]:
openai_api_key = input("Enter your OpenAI API key: ")
os.environ['OPENAI_API_KEY'] = openai_api_key

### Define relevant paths

In [3]:
code_dir = os.getcwd()
root_dir = os.path.dirname(code_dir)
data_dir = os.path.join(root_dir, 'data')
text_dir = os.path.join(root_dir, 'transcripts')

### Define `DialogueAgent` and `DialogueSimulator` classes

The `DialogueAgent` is a simple wrapper for the `ChatOpenAI`; stores the message history rom a `dialogue_agent`'s point of view by concatenating the messages as strings. It uses two methods: 

- `send()` uses the chat model on the message history and returns a message string.
- `receive()` incorporates the message attributed to a specific name into the message history.

The `DialogueSimulator` class takes a list of agents and performs the following: 

1. selects the next stpeaker 
2. calls on the next speaker to send a message 
3. broadcasts the message to all other agents
4. increments time by updating a step counter

In [4]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates or interjects with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

### `BiddingDialogueAgent` subclass

The `BiddingDialogueAgent` has a `bid()` based on message history that factors into the selection and rotation of speakers.

In [5]:
class BiddingDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,
        system_message: SystemMessage,
        bidding_template: PromptTemplate,
        model: ChatOpenAI,
    ) -> None:
        super().__init__(name, system_message, model)
        self.bidding_template = bidding_template

    def bid(self) -> str:
        """
        Asks the chat model to output a bid to speak
        """
        prompt = PromptTemplate(
            input_variables=["message_history", "recent_message"],
            template=self.bidding_template,
        ).format(
            message_history="\n".join(self.message_history),
            recent_message=self.message_history[-1],
        )
        bid_string = self.model([SystemMessage(content=prompt)]).content
        return bid_string

### Output parser for bids

In [6]:
class BidOutputParser(RegexParser):
    def get_format_instructions(self) -> str:
        return "Your response should be an integer delimited by angled brackets."


bid_parser = BidOutputParser(
    regex=r"<(\d+)>", output_keys=["bid"], default_output_key="bid"
)

### Define agents and dialogue characteristics
1. Define a list of agents 
2. Elaborate on a description for the simulation 
3. Set word limit for responses

In [7]:
agent_names = ["Overthrown Government", "Democratic Opposition", "Military Junta"]

game_description = f"""The nation of Pacifica has been torn apart after years of civil war that culminated in a violent military coup. The new leaders of Pacifica have faced immense international pressure to reckon with the atrocities that took place throughout the conflict. In an unprecedented act, they established the Commission for National Reflection (CNR). The CNR is an official truth commission tasked with investigating human rights abuses, documenting an impartial and comprehensive historical record from all sides, and facilitating national reconciliation. Political elites who participated in the conflict, which include leaders from the {', '.join(agent_names)} have been invited to piece together a detailed account of the events leading up to this critical juncture in Pacifica's recent history. By engaging these groups of political elites in dialogue with one another, the CNR seeks to uncover what happened in order to explore the decision-making processes that took place throughout the conflict and prevent this from happening again.
Three groups of political elites that were involved in the conflict will engage in dialogue with one another: {', '.join(agent_names)}. Their collective goal is to create a detailed and accurate historical record of events that represent the strategic considerations and actions of each group in the days immediately leading up to the violent military coup in Pacifica. 
By the end of the dialogue, a representative from one of the groups will summarize their detailed record of what happened on each day of the conflict, one day at a time. 
While each group will represent their perspectives and decision-making processes during this critical period, it is crucial for each group to strive for honesty and acknowledge any wrongdoings committed by their side. Presenting idealized or sanitized versions of the conflict would be counterproductive to facilitating genuine national reconciliation. 
All groups must approach this dialogue process with full transparency and acknowledge their own contributions to the escalation of violence during this conflict. Failure to confront the atrocities and human rights violations that occurred risks perpetuating cycles of violence and instability in Pacifica's future. 
"""

word_limit = 100

### Generate system message templates for agents 

The system message templates are used for:

- bidding templates that determine the speaker rotation
- temporally adjusted agent system messages that explicitly specify which day the agent should be talking about

In [8]:
agent_descriptor_system_message = SystemMessage(
    content="Provide an analysis of each group of political elites. Make sure to elaborate on a set of goals, beliefs, and behavioral tendencies that will help guide each group through the dialogue facilitated by the CNR."
)

def generate_agent_description(agent_name):
    agent_specifier_prompt = [
        agent_descriptor_system_message, 
        HumanMessage(
        content=f"""{game_description}
        Respond with a concise description of the {agent_name} in {word_limit} words.
        Focus exclusively on their goals, beliefs, and typical behaviors, differentiating between their approach during a conflict and in the current day. Specifically:
        - Address the {agent_name} directly in second person. 
        - State their goals. 
        - Outline their primary objectives and foundational beliefs.
        - Describe their typical actions or behaviors, noting any changes from the time of conflict to present-day.
        - Note their direct contribution to escalations of violence. 
        - Mention their perspective on and intention toward both of the other political elite groups involved in this dialogue.
        Do not use idealized, vague, or sanitized descriptions of the {agent_name}.
        Do not use any headers, titles, or introductory labels such as "{agent_name}".
        Clearly indicate whether certain agentistics were true during the conflict or if they apply to the present-day {agent_name}.
        Do not add anything else."""
    ),
    ]
    agent_description = ChatOpenAI(model = 'gpt-4-1106-preview', temperature=0.3)(agent_specifier_prompt).content 
    return agent_description


def generate_agent_header(agent_name, agent_description):
    return f"""{game_description}
    You are the {agent_name}. 
    Your group played a significant role in various aspects of the civil war in Pacifica.
    {agent_description}
    """

# Concatenate agent descriptions and agent headers  
agent_descriptions = [
    generate_agent_description(agent_name) for agent_name in agent_names
]
agent_headers = [
    generate_agent_header(agent_name, agent_description)
    for agent_name, agent_description in zip(
        agent_names, agent_descriptions
    )
]

In [9]:
# Save and print agent descriptions
agent_desc_path = os.path.join(text_dir, 'agent_descriptions.txt')

with open(agent_desc_path, 'w') as f:
    for (
        agent_name,
        agent_description,
    ) in zip(
        agent_names,
        agent_descriptions,
    ):
        f.write(f"\n\n{agent_name} Description:")
        f.write(f"\n{agent_description}")
f.close()

f = open(agent_desc_path, 'r')
file_contents = f.read()
f.close()

print (file_contents)



Overthrown Government Description:
You, the Overthrown Government, once sought to maintain national unity and enforce your vision of order, often prioritizing stability over democratic freedoms. Your goals were to consolidate power and implement policies you believed would benefit the nation, even if it meant suppressing dissent. During the conflict, you were known to take aggressive stances against opposition, sometimes resulting in human rights abuses and contributing to the escalation of violence. Now, in the present day, you aim to regain legitimacy and political influence, showing a willingness to engage in dialogue and reconciliation. You view the Democratic Opposition with skepticism, believing they undermined your authority, while you consider the Military Junta as usurpers who betrayed you. Your current behavior is more conciliatory, as international and domestic pressures have necessitated a shift from your previously authoritarian actions.

Democratic Opposition Descriptio

### Generate bidding system message

In [10]:
def generate_agent_bidding_template(agent_header):
    bidding_template = f"""{agent_header}

```
{{message_history}}
```

On the scale of 1 to 10, where 1 is "not at all" and 10 is "extremely", to what extent do you believe that this statement is referring to your strategies and actions?

```
{{recent_message}}
```

{bid_parser.get_format_instructions()}
Do nothing else.
    """
    return bidding_template


agent_bidding_templates = [
    generate_agent_bidding_template(agent_header)
    for agent_header in agent_headers
]

In [11]:
for agent_name, bidding_template in zip(
    agent_names, agent_bidding_templates
):
    print(f"{agent_name} Bidding Template:")
    print(bidding_template)

Overthrown Government Bidding Template:
The nation of Pacifica has been torn apart after years of civil war that culminated in a violent military coup. The new leaders of Pacifica have faced immense international pressure to reckon with the atrocities that took place throughout the conflict. In an unprecedented act, they established the Commission for National Reflection (CNR). The CNR is an official truth commission tasked with investigating human rights abuses, documenting an impartial and comprehensive historical record from all sides, and facilitating national reconciliation. Political elites who participated in the conflict, which include leaders from the Overthrown Government, Democratic Opposition, Military Junta have been invited to piece together a detailed account of the events leading up to this critical juncture in Pacifica's recent history. By engaging these groups of political elites in dialogue with one another, the CNR seeks to uncover what happened in order to explore 

### Use LLMs to initiate and conclude the discussion 

`temperature` parameters for the `opening_statement` and `closing_statement` are set to 1.0 to allow for more randomness/diversity in the responses. In putting together this prototype, I found that the prompts for generating these statements are fairly straightforward and adding some randomness to generated responses does not come at the cost of derailing the simulation.

In [12]:
opening_prompt = [
    SystemMessage(content="Initiate the dialogue and state the formal rules of the game. The goal of this dialogue is to create a detailed and accurate day-by-day record of events that represent the strategic considerations and actions of each group in the days leading up to the violent military coup in Pacifica."),
    HumanMessage(
        content=f"""{game_description}
        You are the facilitator from the Commission for National Recognition. You are initializing dialogue in the aftermath of a military coup between representatives from the Overthrown Government, Military Junta, and Democratic Opposition.
        Reply with the opener for this dialogue in 100 words or less. 
        Speak directly to representatives from the Overthrown Government, Military Junta, and Democratic Opposition.
        Do not add anything else."""
    ),
]
opening_statement = ChatOpenAI(model = 'gpt-4-1106-preview', temperature=1.0)(opening_prompt).content

closing_prompt = [
    SystemMessage(content="You must be impartial towards all the representatives from the Overthrown Government, Military Junta, and Democratic Opposition, regardless of what was shared in the dialogue."),
    HumanMessage(
        content=f"""{game_description}
        You are the facilitator from the Commission for National Recognition. You are concluding the dialogue after representatives from the {', '.join(agent_names)} shared details on what happened in the days leading up to the military coup.
        Thank the representatives from {', '.join(agent_names)} for sharing their accounts of what happened. 
        Suggest what the next step is towards reconciliation. 
        Reply with the closing statement for this dialogue in 100 words or less. 
        Speak directly to representatives from the Overthrown Government, Military Junta, and Democratic Opposition.
        Do not add anything else."""
    ),
]

closing_statement = ChatOpenAI(model = 'gpt-4-1106-preview', temperature=1.0)(closing_prompt).content


### Define the speaker selection function 

In [13]:
@tenacity.retry(
    stop=tenacity.stop_after_attempt(2),
    wait=tenacity.wait_none(),  # No waiting time between retries
    retry=tenacity.retry_if_exception_type(ValueError),
    before_sleep=lambda retry_state: print(
        f"ValueError occurred: {retry_state.outcome.exception()}, retrying..."
    ),
    retry_error_callback=lambda retry_state: 0,
)  # Default value when all retries are exhausted
def ask_for_bid(agent) -> str:
    """
    Ask for agent bid and parses the bid into the correct format.
    """
    bid_string = agent.bid()
    bid = int(bid_parser.parse(bid_string)["bid"])
    return bid

In [14]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    bids = []
    for agent in agents:
        bid = ask_for_bid(agent)
        bids.append(bid)

    # randomly select among multiple agents with the same bid
    max_value = np.max(bids)
    max_indices = np.where(bids == max_value)[0]
    idx = np.random.choice(max_indices)

    dialogue_transcript.append("Bids:")
    for i, (bid, agent) in enumerate(zip(bids, agents)):
        dialogue_transcript.append(f"\t{agent.name} bid: {bid}")
        if i == idx:
            selected_name = agent.name
    dialogue_transcript.append(f"Selected: {selected_name}")
    dialogue_transcript.append("\n")
    return idx

### Define agent system messages

Agent system messages can be customized based on temporal factors to account for the following:
- `day_count` decreases to inform the agents which day they should be talking about. 
- The first agent does not reply to the previous speaker's comment.
- All other agents are prompted to respond to the previous statement and directly address the agent that spoke before them. 

In [17]:
def generate_agent_system_message(agent_name, agent_header, day_count=None, first_day=False, other_days=True, closing_statement=False): 
    content = f"""{agent_header}
    You will present the strategy and actions of {agent_name}.
    You will speak in the first person, strictly from {agent_name}'s perspective.
    Do not repeatedly paraphrase anything you say.
    DO NOT CHANGE ROLES!
    Do not speak from the perspective of anyone else.
    Speak only from the perspective of {agent_name}.
    Stop speaking the moment you finish speaking from your perspective.
    Do not generate any replies from other agents.
    Never forget to keep your response to {word_limit} words!"""

    # Conditionally add to the content string based on time_period
    if day_count:
        content += f"""\nYou will provide detailed information on what happened {day_count} days before the coup.
        Include any relevant information on events that took place on that specific day."""

    # Conditionally add to the content string if it is the first day 
    if first_day: # In a future iteration, this prompt would only apply to the first agent that speaks. 
        content += """\nYou are the first agent to speak after the CNR facilitator initiated the dialogue. 
        Do not address, reply to, or reference any of the other agents by name or role. 
        Speak only about your own actions and perspectives without engaging the other agents directly."""

    # Conditionally add the following to the content string otherwise
    if other_days:
        content += """\n In addition to giving your own statement, respond to a previous statement. 
        Talk to the speaker of the statement you are responding to directly.
        Do not ask questions. 
        Phrase your response in sentence form.""" # The restriction on questions can be adjusted in a more dynamic setting.

    # Add additional information for specific points in the dialogue 
    if closing_statement:
        content += """\n Provide a closing statement for your contribution to this discussion. 
        Reflect and respond to what has been discussed.
        Be strategic in how you phrase your closing statement.
        The truth commission will make decisions on national reconciliation based on what was presented in this discussion."""

    # Final addition to the content string
    content += "\nDo not add anything else."

    # Return the SystemMessage object with the built content
    return SystemMessage(content=content)

def remove_selected_agent(agents, selected_name):
    return [agent for agent in agents if agent.name != selected_name]

### Dialogue generation

- Specifies the starting point for agents' recollection (how many days before the coup)
- Initiates the dialogue with the CNR representative 
- Adds in dialogue from the CNR representative at the beginning of discussing each day to ensure generation of a a historical record/timeline of events 
- Generates custom agent system messages for each day that is discussed 
- Sets `temperature` parameter to 0.6 to conservatively limit the generation of text that is too random to be realistic 
- Sets `frequency` parameter to 0.6 to control for some repetitiveness

In [24]:
days_before_coup = first_day = 7

# Initiate the dialogue
dialogue_transcript = [] 
dialogue_agents = []

simulator = DialogueSimulator(agents=[], selection_function=select_next_speaker)
simulator.reset()
simulator.inject("CNR Representative", opening_statement)
dialogue_transcript.append(f"(CNR Representative): {opening_statement}")
dialogue_transcript.append("\n")

# Recount the first day 
before_coup = f"{days_before_coup} days before the military coup"
cnr_message = f"Tell us what happened {before_coup}."

simulator.inject("CNR Representative", {cnr_message})
dialogue_transcript.append(f"(CNR Representative): {cnr_message}")
dialogue_agents = []
for i, (agent_name, agent_header, bidding_template) in enumerate(zip(
    agent_names, agent_headers, agent_bidding_templates
)):
    system_message_tmp = generate_agent_system_message(agent_name, agent_header, days_before_coup, first_day=True, other_days=False)

    agent = BiddingDialogueAgent(
        name=agent_name,
        system_message=system_message_tmp,
        model=ChatOpenAI(
            model = 'gpt-4-1106-preview', 
            temperature=0.5,
            model_kwargs={
                "frequency_penalty": 0.6
            }
        ),
        bidding_template=bidding_template,
    )
    dialogue_agents.append(agent)
    simulator.agents = dialogue_agents.copy()

remaining_agents = dialogue_agents.copy()

for _ in range(3):
    name, message = simulator.step()
    dialogue_transcript.append(f"({name}): {message}")
    dialogue_transcript.append("\n")

    selected_agent = next((agent for agent in remaining_agents if agent.name == name), None)
    if selected_agent:
        remaining_agents.remove(selected_agent)

    simulator.agents = remaining_agents.copy()

# Recount remaining days before the coup  
while days_before_coup > 0:
    days_before_coup -= 1
    before_coup = f"{days_before_coup} days before the military coup"  # Update the string to reflect the current value
    cnr_message = f"Tell us what happened {before_coup}."  # Update the message if necessary
    simulator.inject("CNR Representative", {cnr_message})
    dialogue_transcript.append(f"(CNR Representative): {cnr_message}")
    dialogue_agents = []
    for agent_name, agent_header, bidding_template in zip(
        agent_names, agent_headers, agent_bidding_templates
    ):
        system_message_tmp = generate_agent_system_message(agent_name, agent_header, days_before_coup)
        agent = BiddingDialogueAgent(
            name=agent_name,
            system_message=system_message_tmp,
            model=ChatOpenAI(
                model='gpt-4-1106-preview', 
                temperature=0.5, 
                model_kwargs={
                    "frequency_penalty": 0.6
                }
            ),
            bidding_template=bidding_template,
        )
        dialogue_agents.append(agent)
        simulator.agents = dialogue_agents.copy()

    remaining_agents = dialogue_agents.copy()

    for _ in range(3):  
        name, message = simulator.step()
        dialogue_transcript.append(f"({name}): {message}")
        dialogue_transcript.append("\n")

        selected_agent = next((agent for agent in remaining_agents if agent.name == name), None)
        if selected_agent:
            remaining_agents.remove(selected_agent)

        simulator.agents = remaining_agents.copy()
    
# Day of the coup 
after_coup = "on the day of the military coup"
dialogue_agents = []
for agent_name, agent_header, bidding_template in zip(
    agent_names, agent_headers, agent_bidding_templates
):
    system_message_tmp = generate_agent_system_message(agent_name, agent_header, after_coup)
    agent = BiddingDialogueAgent(
        name=agent_name,
        system_message=system_message_tmp,
        model=ChatOpenAI(
            model = 'gpt-4-1106-preview', 
            temperature=0.5, 
            model_kwargs={
                "frequency_penalty": 0.6
            }
        ),
        bidding_template=bidding_template,
    )
    dialogue_agents.append(agent)
    simulator.agents = dialogue_agents.copy()

remaining_agents = dialogue_agents.copy()

for _ in range(3):  
    name, message = simulator.step()
    dialogue_transcript.append(f"({name}): {message}")
    dialogue_transcript.append("\n")

    selected_agent = next((agent for agent in remaining_agents if agent.name == name), None)
    if selected_agent:
        remaining_agents.remove(selected_agent)

    simulator.agents = remaining_agents.copy()

# Closing statements 
dialogue_agents = []
for agent_name, agent_header, bidding_template in zip(
    agent_names, agent_headers, agent_bidding_templates
):
    system_message_tmp = generate_agent_system_message(agent_name, agent_header, other_days=False, closing_statement=True)
    agent = BiddingDialogueAgent(
        name=agent_name,
        system_message=system_message_tmp,
        model=ChatOpenAI(
            model = 'gpt-4-1106-preview', 
            temperature=0.5, 
            model_kwargs={
                "frequency_penalty": 0.6
            }
        ),
        bidding_template=bidding_template,
    )
    dialogue_agents.append(agent)
    simulator.agents = dialogue_agents.copy()

remaining_agents = dialogue_agents.copy()

for _ in range(3):  
    name, message = simulator.step()
    dialogue_transcript.append(f"({name}): {message}")
    dialogue_transcript.append("\n")

    selected_agent = next((agent for agent in remaining_agents if agent.name == name), None)
    if selected_agent:
        remaining_agents.remove(selected_agent)

    simulator.agents = remaining_agents.copy()


# Conclude the dialogue 
simulator.inject("CNR Representative", closing_statement)
dialogue_transcript.append(f"(CNR Representative): {closing_statement}")

In [25]:
for line in dialogue_transcript: 
    print(line)

(CNR Representative): Esteemed representatives from the Overthrown Government, Military Junta, and Democratic Opposition, we convene today under the auspices of the Commission for National Reflection to engage in a crucial dialogue. We strive to craft a precise, day-by-day chronicle of events leading up to the coup, exploring each group's strategic considerations and actions. Your candor and commitment to transparency are vital in our collective pursuit of truth and reconciliation for Pacifica. Let us commence with openness and the shared goal of laying the foundation for a peaceful future.


(CNR Representative): Tell us what happened 7 days before the military coup.
Bids:
	Overthrown Government bid: 7
	Democratic Opposition bid: 8
	Military Junta bid: 8
Selected: Military Junta


(Military Junta): Seven days before the coup, we, the Military Junta, were finalizing our strategic plans to seize control of Pacifica. Recognizing the deteriorating state of national security and rampant co

In [26]:
# Save dialogue transcript in a plain text file 
transcript_path = os.path.join(text_dir, 'dialogue_transcript.txt')

with open(transcript_path, 'w') as f:
    for line in dialogue_transcript:
        print(line, file = f)

f.close()

In [28]:
# Convert the dialogue transcript into a data frame for future evaluations
dialogue_data = []

for line in dialogue_transcript:
    # Check if the line is a statement by looking for the pattern "(Speaker): Statement"
    if line.startswith('(') and '):' in line:
        speaker, statement = line.split('):', 1)
        speaker = speaker[1:]  # Remove the opening parenthesis
        statement = statement.strip()  # Remove leading and trailing whitespace
        dialogue_data.append({"speaker": speaker, "statement": statement})

# Convert the list of dictionaries into a DataFrame
dialogue_df = pd.DataFrame(dialogue_data)

# Reorder columns 
cols = dialogue_df.columns.tolist()
cols.insert(0, cols.pop(-1))
dialogue_df = dialogue_df[cols]

# Save to csv 
dialogue_df.to_csv(os.path.join(data_dir, 'dialogue_transcript.csv'))

### Acknowledgments 

This prototype used the [LangChain](https://python.langchain.com/docs/get_started/introduction) framework to develop this dialogue powered by language models. 

The text was generated using the [OpenAI API](https://platform.openai.com/docs/overview). 